In [ ]:
import geopandas as gpd

In [ ]:
schools = gpd.read_file("data/nyc/SchoolPoints_APS_2024_08_28 (1)/SchoolPoints_APS_2024_08_28.shp")
subways = gpd.read_file("data/nyc/nyc_subway_entrances/nyc_subway_entrances.shp")
bike_paths = gpd.read_file("data/nyc/New York City Bike Routes_20241223.geojson")
neighborhoods = gpd.read_file("https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson")
parks = gpd.read_file("data/nyc/Parks Properties_20241223.geojson")

In [ ]:
schools = schools.to_crs("EPSG:3857")
subways = subways.to_crs("EPSG:3857")
bike_paths = bike_paths.to_crs("EPSG:3857")
neighborhoods = neighborhoods.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

In [ ]:
sindex = neighborhoods.sindex

In [ ]:
schools_in_neighborhoods = gpd.sjoin(schools, neighborhoods, how="inner", predicate="within")

In [ ]:
schools_in_neighborhoods.head()

In [ ]:
# Aggregate the results: count schools per neighborhood and include geometry

schools_count_by_neighborhood = schools_in_neighborhoods.groupby('neighborhood').agg(
    school_count=('index_right', 'count')
).reset_index()

In [ ]:
schools_count_by_neighborhood

In [ ]:
# Merge with the neighborhoods dataframe to preserve neighborhood geometry
schools_count_by_neighborhood = neighborhoods.merge(
    schools_count_by_neighborhood, on='neighborhood', how='left'
).fillna({'school_count': 0})

# Convert to a GeoDataFrame
schools_count_by_neighborhood = gpd.GeoDataFrame(schools_count_by_neighborhood, geometry='geometry', crs=neighborhoods.crs)


In [ ]:
schools_count_by_neighborhood